In [8]:
import orjson
import pendulum
import pandas as pd
import numpy as np
import concurrent.futures as cf
from toolkit.fileutils import Fileutils
from toolkit.utilities import Utilities
import user


# user preferences
Y_PERC = 0.03
T_PERC = 0.01
MIN_PRC = 100
MAX_PRC = 10000
GAPITAL = 100000
TRADE_DAY_TMINUS = 1
ACCOUNT = "HARSHITBONI"
BUFFER = 0.01

futil = Fileutils()
lst_dohlcv = ["dtime", "o", "h", "l", "c", "v"]
df_empty = pd.DataFrame()
fpath = "../../../"
dpath = fpath + "data/"
pd.options.mode.chained_assignment = None
tutils = Utilities()
executor = cf.ThreadPoolExecutor()

In [28]:
csvfile = dpath + "4_today_open.csv"
df = pd.read_csv(csvfile, header=0)
df['perc'] = np.where(df['dir'] == 1, (df['open'] - df['pdh']) /
                              df['pdh'], (df['pdl'] - df['open']) / df['open'])
        
is_perc_grt = df['perc'] > T_PERC
df = df[~is_perc_grt]

print("marking GAP trades for Truth")
str_perc = f"perc>0&perc<={T_PERC}"
df['is_gap'] = df.eval(str_perc)
print(df.tail())


print("sorting the perc for breakout trades")
# df['perc'] = df['perc'].replace([np.inf, -np.inf], np.nan)
df['perc'] = df['perc'].round(4)        
df.sort_values(by='perc', ascending=False, inplace=True)
df = df.reset_index(drop=True)
# df['perc'] = df['perc'].round(3).sort_values().reset_index(drop=True)


print("finding max and min to create columns")
df['min'] = df[['open', 'pdl']].min(axis=1)
df['max'] = df[['open', 'pdh']].max(axis=1)

lst_cols_to_drop = ['eod', 'pdo', 'pdc', 'yday_perc']
print(f"dropping unwanted columns {lst_cols_to_drop}")
df.drop(columns=lst_cols_to_drop, inplace=True)

csvfile = dpath + "5_trim.csv"
print(f"saving to {csvfile}")
df.to_csv(csvfile, index=False)
df


marking GAP trades for Truth
            symbol  symboltoken       pdh       pdo       pdl      pdc  \
342   ZFCVINDIA-EQ        16915  12247.95  11964.60  11932.05  12181.0   
343        ZEEL-EQ         3812    179.90    179.60    176.50    177.6   
344  ZENSARTECH-EQ         1076    389.95    384.80    380.55    382.4   
345   ZYDUSWELL-EQ        17635   1505.25   1505.00   1475.20   1483.0   
346      ECLERX-EQ        15179   1720.00   1709.75   1681.05   1681.2   

                  eod  dir  yday_perc     open      perc  is_gap  
342  2023-06-29 15:30   -1   0.026475  12177.0 -0.020116   False  
343  2023-06-29 15:30    1   0.019263    177.8 -0.011673   False  
344  2023-06-29 15:30    1   0.024701    383.2 -0.017310   False  
345  2023-06-29 15:30    1   0.020370   1489.8 -0.010264   False  
346  2023-06-29 15:30    1   0.023170   1696.8 -0.013488   False  
sorting the perc for breakout trades
finding max and min to create columns
dropping unwanted columns ['eod', 'pdo', 'pdc', '

,symbol,symboltoken,pdh,pdl,dir,open,perc,is_gap,min,max
0,HDFC-EQ,1330,2785.00,2761.00,1,2810.00,0.0090,True,2761.00,2810.00
1,EASEMYTRIP-EQ,2792,42.70,41.85,-1,41.50,0.0084,True,41.50,42.70
2,KNRCON-EQ,15283,242.40,237.50,1,244.00,0.0066,True,237.50,244.00
3,AEGISCHEM-EQ,40,325.95,322.50,1,327.85,0.0058,True,322.50,327.85
4,COROMANDEL-EQ,739,945.00,933.50,1,950.10,0.0054,True,933.50,950.10
...,...,...,...,...,...,...,...,...,...,...
337,CCL-EQ,11452,666.95,648.70,-1,669.00,-0.0303,False,648.70,669.00
338,NYKAA-EQ,6545,149.75,147.00,1,145.00,-0.0317,False,145.00,149.75
339,CARBORUNIV-EQ,595,1193.00,1160.55,-1,1199.00,-0.0321,False,1160.55,1199.00
340,ATGL-EQ,6066,658.90,643.00,-1,664.60,-0.0325,False,643.00,664.60
